In [1]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline


init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

In [3]:
datafile = "C:/Users/Adithya/Desktop/BBCHindi(English).csv"
datafile2 = "C:/Users/Adithya/Desktop/BBCHindi.csv"
tweets = pd.read_csv(datafile2, encoding='latin1')
df = pd.read_csv(datafile, encoding='latin1')
tweets = tweets.rename(columns={'date':'Time','text':'Tweet'})
tweets['Tweet'] = list(df['tweet'])
tweets = tweets.assign(Time=pd.to_datetime(tweets['Time']))
print("Number of tweets: ",len(tweets['Tweet']))
tweets.head()

Number of tweets:  14400


,Time,username,to,replies,retweets,favorites,Tweet,geo,mentions,hashtags,id,permalink
0,2020-09-14 16:04:19,BBCHindi,NaN,7,15,228,Is it Oman's turn to join hands with Israel?,NaN,NaN,NaN,1305537875803545600,https://twitter.com/BBCHindi/status/1305537875...
1,2020-09-14 15:49:00,BBCHindi,NaN,4,15,334,NDA's Harivansh Narayan re-elected as Deputy C...,NaN,NaN,NaN,1305534021103910913,https://twitter.com/BBCHindi/status/1305534021...
2,2020-09-14 13:28:32,BBCHindi,NaN,9,37,270,Cartoon: How will Hindi develop like this?,NaN,NaN,NaN,1305498668649385986,https://twitter.com/BBCHindi/status/1305498668...
3,2020-09-14 13:10:09,BBCHindi,NaN,10,31,273,Does the second wife have any right in the hus...,NaN,NaN,NaN,1305494042793385985,https://twitter.com/BBCHindi/status/1305494042...
4,2020-09-14 12:47:44,BBCHindi,NaN,9,51,219,The story of a Muslim couple who helped build ...,NaN,NaN,NaN,1305488402490384385,https://twitter.com/BBCHindi/status/1305488402...


In [4]:
tweets['Time'] = pd.to_datetime(tweets['Time'], format='%y-%m-%d %H:%M:%S')
tweetsT = tweets['Time']
trace = go.Histogram(
x=tweetsT,
marker=dict(
color='blue'
),
opacity=0.75
)
layout = go.Layout(
title='Tweet Activity Over Years',
height=450,
width=1200,
xaxis=dict(
title='Month and year'
),
yaxis=dict(
title='Tweet Quantity'
),
bargap=0.2,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [5]:
corpus=[]
a=[]
for i in range(len(tweets['Tweet'])):
    a=tweets['Tweet'][i]
    corpus.append(a)
corpus[0:10]

["Is it Oman's turn to join hands with Israel?",
 "NDA's Harivansh Narayan re-elected as Deputy Chairman of Rajya Sabha",
 'Cartoon: How will Hindi develop like this?',
 "Does the second wife have any right in the husband's property?",
 'The story of a Muslim couple who helped build several temples in Assam Video: Dilip Kumar Sharma, Jorhat Editing for BBC Hindi: Devashish',
 'Controversies related to IPL that you will not forget',
 'Cartoon: Distancing from Corona!',
 'India-China tension: What is China softening its attitude',
 'Corona to 17 MPs on the first day of the Parliament session.',
 "Today's cartoon: How will Hindi develop like this?"]

In [6]:
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "C:\Users\Adithya\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [7]:
import nltk
nltk.download('stopwords')
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adithya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-05-22 14:13:46,042 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2023-05-22 14:13:46,243 : INFO : adding document #10000 to Dictionary(19995 unique tokens: ['hands', 'israel?', 'join', "oman's", 'turn']...)
2023-05-22 14:13:46,356 : INFO : built Dictionary(26028 unique tokens: ['hands', 'israel?', 'join', "oman's", 'turn']...) from 14400 documents (total 143987 corpus positions)
2023-05-22 14:13:46,356 : INFO : Dictionary lifecycle event {'msg': 'built Dictionary(26028 unique tokens: [\'hands\', \'israel?\', \'join\', "oman\'s", \'turn\']...) from 14400 documents (total 143987 corpus positions)', 'datetime': '2023-05-22T14:13:46.356010', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-05-22 1

In [8]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)

2023-05-22 14:13:51,913 : INFO : storing corpus in Matrix Market format to C:\Users\Adithya\AppData\Local\Temp\elon.mm
2023-05-22 14:13:51,929 : INFO : saving sparse matrix to C:\Users\Adithya\AppData\Local\Temp\elon.mm
2023-05-22 14:13:51,929 : INFO : PROGRESS: saving document #0
2023-05-22 14:13:51,953 : INFO : PROGRESS: saving document #1000
2023-05-22 14:13:51,993 : INFO : PROGRESS: saving document #2000
2023-05-22 14:13:52,018 : INFO : PROGRESS: saving document #3000
2023-05-22 14:13:52,059 : INFO : PROGRESS: saving document #4000
2023-05-22 14:13:52,108 : INFO : PROGRESS: saving document #5000
2023-05-22 14:13:52,149 : INFO : PROGRESS: saving document #6000
2023-05-22 14:13:52,190 : INFO : PROGRESS: saving document #7000
2023-05-22 14:13:52,239 : INFO : PROGRESS: saving document #8000
2023-05-22 14:13:52,280 : INFO : PROGRESS: saving document #9000
2023-05-22 14:13:52,321 : INFO : PROGRESS: saving document #10000
2023-05-22 14:13:52,345 : INFO : PROGRESS: saving document #11000
2

In [9]:
tfidf = models.TfidfModel(corpus)

2023-05-22 14:13:55,586 : INFO : collecting document frequencies
2023-05-22 14:13:55,586 : INFO : PROGRESS: processing document #0
2023-05-22 14:13:55,626 : INFO : PROGRESS: processing document #10000
2023-05-22 14:13:55,749 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 14400 documents and 26028 features (141563 matrix non-zeros)', 'datetime': '2023-05-22T14:13:55.749426', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'initialize'}


In [10]:
corpus_tfidf = tfidf[corpus]

In [11]:
total_topics = 10
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf]

2023-05-22 14:13:59,544 : INFO : using symmetric alpha at 0.1
2023-05-22 14:13:59,544 : INFO : using symmetric eta at 0.1
2023-05-22 14:13:59,552 : INFO : using serial LDA version on this node
2023-05-22 14:13:59,730 : INFO : running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 14400 documents, updating model once every 2000 documents, evaluating perplexity every 14400 documents, iterating 50x with a convergence threshold of 0.001000
2023-05-22 14:13:59,730 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2023-05-22 14:13:59,738 : INFO : PROGRESS: pass 0, at document #2000/14400
2023-05-22 14:14:01,364 : INFO : merging changes from 2000 documents into a model of 14400 documents
2023-05-22 14:14:01,420 : INFO : topic #1 (0.100): 0.010*"people" + 0.006*"corona" + 0.006*"live" + 0.005*"bbc" + 0.005*"say" + 0.005*"singh" + 0.005*"modi" + 0.004*"first" + 0.004*"today's" 

In [11]:
lda.show_topics(total_topics,10)

[(0,
  '0.015*"india" + 0.014*"congress" + 0.012*"statement" + 0.011*"leader" + 0.011*"bjp" + 0.011*"minister" + 0.011*"opinion" + 0.009*"review" + 0.009*"said" + 0.008*"shiv"'),
 (1,
  '0.025*"cartoon" + 0.025*"(video:" + 0.023*"cartoon:" + 0.020*"police" + 0.018*"supreme" + 0.017*"court" + 0.017*"#bbchindicartoons" + 0.010*"case" + 0.009*"kashmir" + 0.009*"delhi"'),
 (2,
  '0.015*"people" + 0.013*"corona" + 0.008*"due" + 0.008*"one" + 0.008*"report" + 0.008*"ground" + 0.007*"many" + 0.007*"(pc:" + 0.006*"killed" + 0.006*"telling"'),
 (3,
  '0.028*"minister" + 0.022*"modi" + 0.021*"prime" + 0.015*"narendra" + 0.012*"imran" + 0.012*"khan" + 0.011*"corona" + 0.011*"pakistan" + 0.011*"amit" + 0.010*"nehru"'),
 (4,
  '0.028*"press" + 0.026*"review" + 0.015*"indian" + 0.011*"girl" + 0.010*"sportswoman" + 0.010*"oil" + 0.008*"imran" + 0.007*"muslims" + 0.006*"state" + 0.006*"article"'),
 (5,
  '0.025*"video:" + 0.017*"corona" + 0.014*"people" + 0.010*"bbc" + 0.007*"salman" + 0.007*"india" +

In [12]:
data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}

In [13]:
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(10, 196)


In [14]:
df_lda

,video:,kashmir,people,like,ground,school,elections:,report,corona,riyaz,...,change,viral,target,whole,naveen,attacks,temple,mahatma,missing,problems
0,0.010786,0.010765,0.010739,0.009845,0.009223,0.008487,0.007611,0.007591,0.007384,0.006778,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.005279,0.005399,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.006635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.030539,0.000000,0.018929,0.000000,0.000000,0.000000,0.000000,0.000000,0.009948,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.007048,0.000000,0.000000,0.000000,0.000000,0.000000,0.022734,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022121,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.006123,0.000000,0.000000,0.000000,0.000000,0.000000,0.017674,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.010542,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.010347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005541,0.005167,0.005098,0.00502,0.00441,0.004239,0.004233,0.004201,0.004158,0.004096


In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8     -161.040329  115.565796       1        1  23.500590
0      140.163086  141.863525       2        1  14.600708
3       96.560257 -158.613037       3        1  12.602238
1      -24.305140  176.382904       4        1  11.431788
2      -74.157692   22.565741       5        1   9.798560
4       18.146191  -59.755402       6        1   5.616643
9      -71.541237 -154.253708       7        1   5.612371
5      159.308762  -17.266037       8        1   5.612368
6     -177.576202  -53.297710       9        1   5.612368
7       43.368237   61.715797      10        1   5.612367, topic_info=         Term       Freq      Total Category  logprob  loglift
39     video:  87.000000  87.000000  Default  30.0000  30.0000
554    police  54.000000  54.000000  Default  29.0000  29.0000
298     press  30.000000  30.000000  Default  28.0000  28.0000
52     corona  96.000000  96.000000  Default  27.0000  27.0000
299    review  29.000000  29.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
202     india   5.805731  80.856798  Topic10  -4.9359   0.2464
1535    case:   3.366980   8.768864  Topic10  -5.4808   1.9230
187     said,   3.918023  28.213122  Topic10  -5.3292   0.9060
890   india's   3.722411  19.309910  Topic10  -5.3804   1.2340
256       say   3.548186  46.022984  Topic10  -5.4283   0.3175

[643 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
13160      6  0.682767   #bbchindicartoon
12718      6  0.971435  #bbchindicartoons
18380      6  0.834036   #bbhcindicartoon
13352      6  1.060817  #bbhcindicartoons
6590       2  0.210048       #coronavirus
...      ...       ...                ...
840        5  0.048052                â
840        6  0.048052                â
840        8  0.192209                â
840        9  0.048052                â
840       10  0.288313                â

[1125 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 1, 4, 2, 3, 5, 10, 6, 7, 8])